In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [35]:
pip install pillow

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [3]:
DIRECTORY = r"/Users/pranjalbhadu/Documents/smart-workplace/mask_detection/Face_Mask_Dataset/Train"
CATEGORIES = ["WithMask", "WithoutMask"]

In [4]:
data = []
labels = []

In [5]:
from sklearn.preprocessing import LabelEncoder

In [33]:
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        try:
            image=load_img(img_path, target_size=(224, 224))
            image = img_to_array(image)
            image = preprocess_input(image)
            data.append(image)
            labels.append(category)
        except PIL.UnidentifiedImageError:
                print('pil')

pil


In [35]:
labels[:5]

['WithMask', 'WithMask', 'WithMask', 'WithMask', 'WithMask']

In [36]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [37]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [38]:
trainX = data
trainY = labels

In [50]:
test_data=[]
test_labels=[]

In [51]:
TEST_DIRECTORY = r"/Users/pranjalbhadu/Documents/smart-workplace/mask_detection/Face_Mask_Dataset/Test"

In [52]:
for category in CATEGORIES:
    path = os.path.join(TEST_DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        try:
            image=load_img(img_path, target_size=(224, 224))
            image = img_to_array(image)
            image = preprocess_input(image)
            test_data.append(image)
            test_labels.append(category)
        except PIL.UnidentifiedImageError:
                print('pil')

In [53]:
test_labels = lb.fit_transform(test_labels)
test_labels = to_categorical(test_labels)

In [54]:
testX = np.array(test_data, dtype="float32")
testY = np.array(test_labels)

In [63]:
type(trainY)

numpy.ndarray

In [39]:
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

2022-06-07 09:51:45.658859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [40]:
baseModel

In [41]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [42]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [43]:
for layer in baseModel.layers:
	layer.trainable = False

In [45]:
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [68]:
len(testY)//BS

31

In [72]:
import tensorflow as tf

In [73]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [74]:
H = model.fit(trainX, trainY, batch_size=BS,
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=5,  callbacks=[callback])

Epoch 1/5
538/538 [==============================] - 341s 633ms/step - loss: 0.0228 - accuracy: 0.9942 - val_loss: 0.0198 - val_accuracy: 0.9940
Epoch 2/5
538/538 [==============================] - 396s 737ms/step - loss: 0.0155 - accuracy: 0.9958 - val_loss: 0.0168 - val_accuracy: 0.9950
Epoch 3/5
538/538 [==============================] - 423s 787ms/step - loss: 0.0111 - accuracy: 0.9974 - val_loss: 0.0141 - val_accuracy: 0.9970
Epoch 4/5
538/538 [==============================] - 387s 720ms/step - loss: 0.0085 - accuracy: 0.9980 - val_loss: 0.0133 - val_accuracy: 0.9970
Epoch 5/5
538/538 [==============================] - 375s 697ms/step - loss: 0.0067 - accuracy: 0.9981 - val_loss: 0.0137 - val_accuracy: 0.9950


In [75]:
predIdxs = model.predict(testX, batch_size=BS)

31/31 [==============================] - 19s 579ms/step


In [76]:
predIdxs = np.argmax(predIdxs, axis=1)

In [77]:
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

              precision    recall  f1-score   support

    WithMask       0.99      1.00      0.99       483
 WithoutMask       1.00      0.99      1.00       509

    accuracy                           0.99       992
   macro avg       0.99      1.00      0.99       992
weighted avg       0.99      0.99      0.99       992



In [78]:
model.save("mask_detector.model", save_format="h5")